In [2]:
# TODO: add weighted loss
# TODO: logits scaling

In [3]:
!pip install efficientnet_pytorch
!pip install albumentations

from efficientnet_pytorch import EfficientNet
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from kaggle_datasets import KaggleDatasets
import torch
from torch.utils.data import Dataset, DataLoader
from typing import List
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose,
)
import albumentations
from sklearn import metrics
from tqdm import tqdm

In [4]:
epochs = 10
lr = 1e-5
save_path = ''
batch_size = 10
cover_data_path = '/kaggle/input/alaska2-image-steganalysis/Cover'
test_data_path = '/kaggle/input/alaska2-image-steganalysis/Test'
base_data_path = '/kaggle/input/alaska2-image-steganalysis/'
device = torch.device('cuda:0')
log_every = 100
target_metric = 'loss'

In [5]:
def get_orig_imgs(path):
    return os.listdir(path)

def get_negative_examples(path):
    return [os.path.join(path, img) for img in get_orig_imgs(path)]

def get_positive_examples(base_path, base_data_path:str, orig_images: List[str]):
    folders = ['JMiPOD', 'JUNIWARD', 'UERD']    
    positive_images = []
    
    for folder in folders:
        for img in orig_images:
            positive_images.append(os.path.join(base_path, folder, img))
    
    return positive_images

def strong_aug(p=0.5):
    return albumentations.Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=0.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=0.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),
        ], p=0.3),
        HueSaturationValue(p=0.3),
    ], p=p)

In [6]:
class ImageDataset(Dataset):
    def __init__(self, image_paths: List[str], labels: np.array, augs=None):
        self.labels = labels
        self.image_paths = image_paths
        
        self.tfms = transforms.Compose([transforms.Resize(512), 
                                        transforms.ToTensor()])
        if augs is not None:
            self.augs = augs()
        else:
            self.augs = augs
        
    def __getitem__(self, idx):
        label = self.labels[idx]
        img = self.image_paths[idx]
        image = plt.imread(img)
        image = Image.fromarray(image).convert('RGB')
        
        if self.augs is not None:
            image = self.augs(image=np.array(image))['image']
        else:
            image = np.array(image)
        
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        return {'X': torch.tensor(image), 'Y': label}
    
    def __len__(self):
        return len(self.labels)

In [7]:
orig_imgs = get_orig_imgs(cover_data_path)
test_imgs = get_negative_examples(test_data_path)

negatives = get_negative_examples(cover_data_path)
positives = get_positive_examples(base_data_path, cover_data_path, orig_imgs)
train_paths = negatives + positives
train_labels = [1] * len(positives) + [0] * len(negatives)

train_paths, valid_paths, train_labels, valid_labels = train_test_split(
    train_paths, train_labels, test_size=0.15, random_state=2020)


train_dataset = ImageDataset(train_paths, train_labels, strong_aug)
valid_dataset = ImageDataset(valid_paths, valid_labels, strong_aug)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [8]:
class EfficientNetClassifier(torch.nn.Module):
    def __init__(self, efficient_net_name):
        super(EfficientNetClassifier, self).__init__()
        self.backbone = EfficientNet.from_pretrained(efficient_net_name, num_classes=1)
    
    def forward(self, batch: torch.Tensor) -> torch.tensor:
        # 3, 512, 512
        return self.backbone.forward(batch)

def evaluate_model(val_dataloader: DataLoader, classifier: torch.nn.Module, criterion):
    epoch_valid_loss = []
    all_probs = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(val_dataloader):
            x = batch['X'].to(device)
            labels = batch['Y'].to(device)

            logits = classifier.forward(x)
            loss = criterion(logits, labels.float().unsqueeze(dim=1))
            probs = torch.sigmoid(logits.squeeze())
            
            epoch_valid_loss.append(loss.item())
            all_probs.extend(probs.tolist())
            all_labels.extend(labels.tolist())
            
    loss = np.mean(epoch_valid_loss)
    return all_probs, all_labels, loss


def weighted_auc(y_true, y_valid):
    tpr_thresholds = [0.0, 0.4, 1.0]
    weights =        [       2,   1]
    
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_valid, pos_label=1)
#     print(fpr, len(fpr))
    
    # size of subsets
    areas = np.array(tpr_thresholds[1:]) - np.array(tpr_thresholds[:-1])
    
    # The total area is normalized by the sum of weights such that the final weighted AUC is between 0 and 1.
    normalization = np.dot(areas, weights)
    
    competition_metric = 0
    for idx, weight in enumerate(weights):
        y_min = tpr_thresholds[idx]
        y_max = tpr_thresholds[idx + 1]
        mask = (y_min < tpr) & (tpr < y_max)

        x_padding = np.linspace(fpr[mask][-1], 1, 100)

        x = np.concatenate([fpr[mask], x_padding])
        y = np.concatenate([tpr[mask], [y_max] * len(x_padding)])
        y = y - y_min # normalize such that curve starts at y=0
        score = metrics.auc(x, y)
        submetric = score * weight
        best_subscore = (y_max - y_min) * weight
        competition_metric += submetric
        
    return competition_metric / normalization

def make_submission(model: torch.nn.Module, test_data_path: str, batch_size: int, device: torch.device) -> pd.DataFrame:
    
    images = get_negative_examples(test_data_path)[:100]
    
    dataset = ImageDataset(images, [0] * len(images))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.ones(1)).to(device)
    
    all_probs, all_labels, loss = evaluate_model(dataloader, model, criterion)
    all_preds = (torch.tensor(all_probs, dtype=torch.float32) > 0.5).long()
    
    images_names = get_orig_imgs(test_data_path)[:100]
    return pd.DataFrame({'Id': images_names, 'Label': all_preds})

In [9]:
classifier = EfficientNetClassifier('efficientnet-b1').to(device)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.ones(1)).to(device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=lr)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b1-f1951068.pth" to /root/.cache/torch/checkpoints/efficientnet-b1-f1951068.pth



Loaded pretrained weights for efficientnet-b1


In [12]:
train_history = []
val_history = []

for e in range(epochs):
    epoch_train_loss = []
    epoch_valid_loss = []
    print(f'Running epoch: {e}/{epochs}')
    full_save_path = os.path.join(save_path, f'model_ep{e}.pt')
    
    for idx, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        
        x = batch['X'].to(device)
        labels = batch['Y'].to(device)
            
        logits = classifier.forward(x)
        loss = criterion(logits, labels.float().unsqueeze(dim=1))
        loss.backward()
        optimizer.step()
        
        
        epoch_train_loss.append(loss.item())
        
        if idx % log_every == 0:
            print(f'Epoch: {e}/{epochs}')
            print(f'Batch {idx}/{len(train_dataloader)}: bce_loss: {loss}')
    
    train_history.append({'loss': np.mean(epoch_train_loss)})
    
    val_probs, val_labels, val_loss = evaluate_model(val_dataloader, classifier, criterion)
    val_preds =  (torch.tensor(val_probs, dtype=torch.float32) > 0.5).long()
    
#     val_weighted_auc = weighted_auc(val_labels, val_probs)
    auc = metrics.roc_auc_score(val_labels, val_preds)
    val_accuracy = metrics.accuracy_score(val_preds, val_labels)
    val_f1_score = metrics.f1_score(val_preds, val_labels)
    
    val_history.append({'epoch': e, 'loss': val_loss, 'accuracy': val_accuracy, 
                        'auc': 0.0, 'f1_score': val_f1_score, 'save_path': full_save_path})
    
    torch.save(classifier.state_dict(), full_save_path)

Running epoch: 0/10
Epoch: 0/10
Batch 0/25500: 0.685379147529602


TypeError: Cannot handle this data type

In [ ]:
best_epoch = min(val_history, key=lambda hist: hist[target_metric])
best_load_path = best_epoch['save_path']

classifier = EfficientNetClassifier('efficientnet-b1').to(device)
classifier.load_state_dict(torch.load(best_load_path))

df = make_submission(classifier, test_data_path, batch_size, device)
df.to_csv('submission.csv')